# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

598

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [9]:
base_url = "http://api.openweathermap.org/data/2.5/weather?"
# set up dataframe
weather_data = pd.DataFrame()
weather_data["lat"] = ''
weather_data['lng'] = ''
weather_data['max temp'] = ''
weather_data['humidity'] = ''
weather_data['cloudiness'] = ''
weather_data['wind speed'] = ''
weather_data['country'] = ''
weather_data['date'] = ''
# loop through cities (only do 10 right now)
for index, city in enumerate(cities[0:10]):
    # set up query url and access
    query_url = f"{base_url}&appid={weather_api_key}&q={city}"
    response = requests.get(query_url).json()
    print(f"Processing city number {index}  |  {city}")
    # try to import data into our dataframe
    try:
        weather_data.loc[index, "lng"] = response['coord']['lon']
        weather_data.loc[index, "lat"] = response['coord']['lat']
        weather_data.loc[index, "max temp"] = response['main']['temp_max']
        weather_data.loc[index, "humidity"] = response['main']['humidity']
        weather_data.loc[index, "cloudiness"] = response['clouds']['all']
        weather_data.loc[index, "wind speed"] = response['wind']['speed']
        weather_data.loc[index, "country"] = response['sys']['country']
        weather_data.loc[index, "date"] = response['dt']
    # if data not found print message
    except:
        print(f"{city} not found... skipping")

Processing city number 0  |  hilo
Processing city number 1  |  goldsboro
Processing city number 2  |  severo-kurilsk
Processing city number 3  |  terra santa
Processing city number 4  |  berlevag
Processing city number 5  |  atambua
Processing city number 6  |  ponta do sol
Processing city number 7  |  mys shmidta
mys shmidta not found... skipping
Processing city number 8  |  bredasdorp
Processing city number 9  |  tuatapere


,lat,lng,max temp,humidity,cloudiness,wind speed,country,date
0,19.7297,-155.09,298.04,78,100,2.57,US,1644102312
1,35.3849,-77.9928,277.56,35,0,2.57,US,1644102967
2,50.6789,156.125,271.39,82,95,4.51,RU,1644103128
3,-2.1042,-56.4869,297.53,94,100,0.66,BR,1644103128
4,70.8578,29.0864,268.95,82,55,11.54,NO,1644102838
5,-9.1061,124.8925,297.88,83,74,0.18,ID,1644103128
6,32.6667,-17.1,292.6,56,9,5.25,PT,1644103054
8,-34.5322,20.0403,293.31,77,83,8.75,ZA,1644103129
9,-46.1333,167.6833,285.83,73,99,2.18,NZ,1644102698


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [10]:
# export to csv
weather_data.to_csv("../output_data/weather_data.csv")
weather_data

,lat,lng,max temp,humidity,cloudiness,wind speed,country,date
0,19.7297,-155.09,298.04,78,100,2.57,US,1644102312
1,35.3849,-77.9928,277.56,35,0,2.57,US,1644102967
2,50.6789,156.125,271.39,82,95,4.51,RU,1644103128
3,-2.1042,-56.4869,297.53,94,100,0.66,BR,1644103128
4,70.8578,29.0864,268.95,82,55,11.54,NO,1644102838
5,-9.1061,124.8925,297.88,83,74,0.18,ID,1644103128
6,32.6667,-17.1,292.6,56,9,5.25,PT,1644103054
8,-34.5322,20.0403,293.31,77,83,8.75,ZA,1644103129
9,-46.1333,167.6833,285.83,73,99,2.18,NZ,1644102698


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression